In [1]:
import os
import numpy as np
import cv2  # Import OpenCV
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from sklearn.model_selection import train_test_split

In [3]:
# Step 1: Data Preparation
data_path = "/Users/rishipal/SynologyDrive/Drive/Personal develop/UNCC Health Informatics/UNCC_Courses/09_Spring 2024/DSBA6165/Group Project/RiceLeafDiseaseImages"
#data_path = "/content/drive/MyDrive/DSBA6165_Group_Project/RiceLeafDiseaseImages"

In [4]:
def load_images_from_folder(folder, target_size=(224, 224)):
    images = []
    labels = []
    for label, class_name in enumerate(os.listdir(folder)):
        class_path = os.path.join(folder, class_name)
        if os.path.isdir(class_path):  # Ensure it's a directory
            for filename in os.listdir(class_path):
                img_path = os.path.join(class_path, filename)
                image = cv2.imread(img_path)  # Read image using OpenCV
                if image is not None:
                    # Resize the image to a consistent size
                    image = cv2.resize(image, target_size)
                    images.append(image)
                    labels.append(label)

    # Ensure all images have the same shape
    max_shape = max(image.shape for image in images)
    for i, image in enumerate(images):
        images[i] = cv2.resize(image, max_shape[:2])

    # Print the shapes of all images
    for image in images:
        print(image.shape)

    return images, labels

# Load images and labels
images, labels = load_images_from_folder(data_path)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [5]:
# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [6]:
# Normalize pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [41]:
# Step 2: Model Definition
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MultiHeadAttention

num_classes = 10  # Define the number of classes

class GraphConvLayer(keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super(GraphConvLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

    def build(self, input_shape):
        self.conv = Conv2D(self.filters, self.kernel_size, activation='relu', padding='same')

    def call(self, inputs):
        return self.conv(inputs)


In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

class GCNWithAttention(keras.Model):
    def __init__(self, num_classes):
        super(GCNWithAttention, self).__init__()
        # Define your layers here
        self.conv1 = GraphConvLayer(64, (3, 3))
        self.conv2 = GraphConvLayer(64, (3, 3))
        self.conv3 = Conv2D(32, (3, 3), activation='relu')
        self.pool = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.fc1 = Dense(128, activation='relu')
        self.fc2 = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        # Define the forward pass
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [43]:
# Create an instance of the model
#self.attention = MultiHeadAttention(num_heads=8, key_dim=64)

gcn_model = GCNWithAttention(num_classes)


In [44]:
# Compile the model
gcn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [45]:
# Train the model
gcn_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 799s 7s/step - accuracy: 0.4450 - loss: 2.8194 - val_accuracy: 0.7218 - val_loss: 0.7291
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 772s 6s/step - accuracy: 0.7774 - loss: 0.5783 - val_accuracy: 0.8672 - val_loss: 0.3993
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 780s 7s/step - accuracy: 0.9203 - loss: 0.2196 - val_accuracy: 0.8862 - val_loss: 0.3910
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 786s 7s/step - accuracy: 0.9656 - loss: 0.0973 - val_accuracy: 0.9336 - val_loss: 0.2541
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 788s 7s/step - accuracy: 0.9790 - loss: 0.0805 - val_accuracy: 0.9305 - val_loss: 0.3838
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 768s 6s/step - accuracy: 0.9783 - loss: 0.0622 - val_accuracy: 0.9536 - val_loss: 0.2526
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 770s 6s/step - accuracy: 0.9956 - loss: 0.0157 - val_accuracy: 0.9484 - val_loss: 0.2843
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 772s 6s/step - accuracy: 0.9990 - loss: 0.0050 - val_accu

In [46]:
 m# Step 5: Model Evaluation
# Evaluate the model on the test set
test_loss, test_accuracy = gcn_model.evaluate(test_images, test_labels)
print("Test Accuracy:", test_accuracy)

38/38 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9741 - loss: 0.1915
Test Accuracy: 0.9671440720558167
